In [ ]:
import numpy as np
import os
import re
from to_nc import to_nc


In [ ]:
import scattering_delays as sd

λ=532e-9
ρ_s=400
r_vals=np.logspace(-7, 0, 71)
v=sd.v_eff(λ, ρ_s)
LUT={}
L_snow=np.zeros_like(r_vals)+np.NaN
for kr, r in enumerate(r_vals):
    # calculate the optical properties
    mu_e, mu_s, mu_a, g = sd.optical_properties_pure_snow(r, λ, ρ_s, LUT=LUT)
    L_snow[kr]=1/((1-g)*mu_s)
L_air=3e8/v*L_snow

L_vs_r = pc.data().from_dict({'r':r_vals,'L':L_air})
L_vs_r.to_h5('L_vs_r.h5')

In [ ]:
import ATM_waveform  as aw

D=aw.fit.data().from_h5('ILATMW1B_20170717_144930.atm6AT5.h5_q2_out.h5')
D.assign(L_scat = np.interp(D.K0, L_vs_r.r, L_vs_r.L), shot_count = D.shot.copy())



In [ ]:
import csv
meta_dict={}
with open('waveform_archive_meta.csv','r') as fh:
    reader = csv.DictReader(fh)
    for row in reader:
        for key, val in row.items():
            if val=='':
                row[key]=None
        meta_dict[row['name']]=row

meta_dict['L_scat']={'name':'L_scat', 'Dimensions':'shot_count','Coordinate':'No',
                     'description': 'Effective distance in air traveled between isotropic scattering events for the best-fitting waveform',
                     'units':'meters',
                     'source_field':'',
                     'group':None,
                     'precision':'6'}


In [ ]:


## test code
import ATM_waveform  as aw
import pointCollection as pc

in_file=        'ILATMW1B_20170717_144930.atm6AT5.h5_q2_out.h5'
out_file ='ATM_gs_'+in_file.split('.h5')[0]+'.nc'


if os.path.isfile(out_file):
    os.remove(out_file)

D=aw.fit.data().from_h5('ILATMW1B_20170717_144930.atm6AT5.h5_q2_out.h5')

import csv
meta_dict={}
with open('waveform_archive_meta.csv','r') as fh:
    reader = csv.DictReader(fh)
    for row in reader:
        for key, val in row.items():
            if val=='':
                row[key]=None
        meta_dict[row['name']]=row

time_re=re.compile('_(\d\d\d\d)(\d\d)(\d\d)')
date=list(time_re.search(D.filename).groups())
meta_dict['time']['units']=f'seconds since {".".join(date)} 00:00:00'

# add in the scattering_time variable
D.assign(L_scat = np.interp(D.K0, L_vs_r.r, L_vs_r.L), shot_count = D.shot.copy())

meta_dict['L_scat']={'name':'L_scat', 'Dimensions':'shot_count','Coordinate':'No',
                     'description': 'Effective distance between isotropic scattering events for the best-fitting waveform, calculated assuming that photons are traveling in air.',
                     'units':'meters',
                     'source_field':'',
                     'group':None,
                     'precision':'6'}

to_nc(D, out_file, replace=True, meta_dict_file=None, 
          meta_dict=meta_dict, group_attrs=None, group='')


In [ ]:
import xarray as xr
ds=xr.open_dataset('ATM_gs_ILATMW1B_20170717_144930.atm6AT5.nc', group='', decode_timedelta=False)
ds

In [ ]:
import datetime
(np.array(ds.time[0:10])-np.datetime64(datetime.datetime(2018, 1, 1))).astype(float)/24/365.25/3600/1.e9+2018

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.hist(np.log10(np.maximum(1.e-7, np.array(ds.sigma).astype(float))), 200);
plt.gca().set_yscale('log')

In [ ]:
np.unique(np.array(ds.sigma).astype(float))